# LCEL

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ.get("OPENAI_API_KEY")


In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


## Simple chain

In [3]:
prompt = ChatPromptTemplate.from_template(
  "tell me a short joke as a pirate about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()


In [4]:
chain = prompt | model | output_parser


In [5]:
chain.invoke({"topic": "oranges"})


'Why did the pirate go to the doctor with a bag of oranges?\n\nBecause he heard he needed to improve his vitamin "sea"!'

## Document retrievers

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch


In [7]:
vectorstore = DocArrayInMemorySearch.from_texts(
  ["harrison worked at Kensho", "bears like to eat honey"],
  embedding = OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


In [8]:
retriever.get_relevant_documents("where did harrison work?")


[Document(page_content='harrison worked at Kensho'),
 Document(page_content='bears like to eat honey')]

In [9]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template) 


How to think about this chain:
- We'll want only the first and only input on the chain to be user question
  - For this we're going to take something that takes a single question and turns into a dictionary with 2 elements `context` and `question`, a `RunnableMap`
- From then we are going to want to fetch relevant context
- Then pass that to a prompt
- Then pass that to the model
- Then pass that to an output parser, to convert the output message into a string

In [10]:
from langchain.schema.runnable import RunnableMap


In [11]:
chain = RunnableMap({
  "context": lambda x: retriever.get_relevant_documents(x["question"]),
  "question": lambda x: x["question"]
}) | prompt | model | output_parser


In [12]:
chain.invoke({"question": "where did harrison work?"})


'Harrison worked at Kensho.'

What happens under the hood:

In [13]:
inputs = RunnableMap(
    {
        "context": lambda x: retriever.get_relevant_documents(x["question"]),
        "question": lambda x: x["question"],
    }
)


In [14]:
inputs.invoke({"question": "where did harrison work?"})


{'context': [Document(page_content='harrison worked at Kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind and OpenAI functions

In [15]:
functions = [
  {
    "name": "weather_search", # name of the function
    "description": "Search for the weather given an airport code", # description of the function
    "parameters": { # parameters of the function
      "type": "object",
      "properties": {
        "airport_code": {
          "type": "string",
          "description": "The airport code to get the weather for"
        }
      },
      "required": ["airport_code"] # required parameters
    },
  }
]


In [16]:
prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "{input}"),
  ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)


In [17]:
runnable = prompt | model


In [18]:
runnable.invoke({"input": "what is the weather in Boston?"})


AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{\n  "airport_code": "BOS"\n}'}})

In [19]:
functions = [
  {
    "name": "weather_search", # name of the function
    "description": "Search for the weather given an airport code", # description of the function
    "parameters": { # parameters of the function
      "type": "object",
      "properties": {
        "airport_code": {
          "type": "string",
          "description": "The airport code to get the weather for"
        }
      },
      "required": ["airport_code"] # required parameters
    },
  },
  {
    "name": "sports_search", 
    "description": "Search for news of recent sports events",
    "parameters": {
      "type": "object",
      "properties": {
        "team_name": {
          "type": "string",
          "description": "The sport to get news for"
        }
      },
      "required": ["team_name"]
    }
  }
]


In [20]:
model = model.bind(functions=functions)


In [21]:
runnable = prompt | model


In [22]:
runnable.invoke({"input": "how did the Red Sox do last night?"})


AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{\n  "team_name": "Red Sox"\n}'}})

## Fallbacks

### A situation where it's going to fail: using an older OpenAI model to generate a valid JSON

In [23]:
from langchain.llms import OpenAI
import json


In [24]:
simple_model = OpenAI(
  temperature=0,
  max_tokens=1000,
  model="text-davinci-001",
)
simple_chain = simple_model | json.loads


In [25]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"


In [26]:
simple_model.invoke(challenge)


'\n\n["The Waste Land","T.S. Eliot","April is the cruelest month, breeding lilacs out of the dead land"]\n\n["The Raven","Edgar Allan Poe","Once upon a midnight dreary, while I pondered, weak and weary"]\n\n["Ode to a Nightingale","John Keats","Thou still unravish\'d bride of quietness, Thou foster-child of silence and slow time"]'

In [27]:
simple_chain.invoke(challenge)


JSONDecodeError: Extra data: line 5 column 1 (char 103)

In [28]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads


In [29]:
chain.invoke(challenge)


{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [30]:
final_chain = simple_chain.with_fallbacks([chain])


In [31]:
final_chain.invoke(challenge)


{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

## Interface that these `runnables` expose

In [32]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke as a pirate about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser


`invoke` is a synchronous method that calls on one input

In [33]:
chain.invoke({"topic": "oranges"})


'Why did the pirate become a citrus enthusiast?\n\nBecause he couldn\'t resist saying, "Arrrrrange you glad I didn\'t say banana?"'

`batch` is a synchronous method that calls on a list of inputs

In [34]:
chain.batch([{"topic": "oranges"}, {"topic": "apples"}])


['Why did the pirate always carry an orange?\n\nBecause it was his "matey"!',
 'Why did the pirate go to the apple orchard?\n\nBecause he heard there were plenty of "arrrrrrrrrrrrrrrrrgh-pples"!']

`stream` returns an iterable that we can loop over and print out the different elements

In [35]:
for t in chain.stream({"topic": "playstation"}):
    print(t)



Why
 did
 the
 pirate
 refuse
 to
 play
 PlayStation
?


Because
 he
 couldn
't
 stand
 the
 "
sea
"
 of
 games
!
 Arr
r
,
 mate
y
!



All these methods have an `asynchronous` counterpart 

In [36]:
response = await chain.ainvoke({"topic": "bears"})
response


'Why did the pirate take up bear hunting?\n\nBecause he wanted to find the "ahoy" in "grizzly"!'